# Lesson 1 Demo 1: Creating a Table with PostgreSQL 

**In this demo we are going to walk through the basics of creating a table in PostgreSQL, inserting rows of data, and doing a simple SQL query to validate the information.** 

**We will use a python wrapper called psycopg2 to run the PostgreSQL queries. This library should be preintalled but in the future to install this library you can run this command in a notebook to install locally:
    
!pip3 install --user psycopg2

**More documentation can be found here:** http://initd.org/psycopg/

**Import the library**

Note: An error might popup after this command has executed. If it does, read it carefully before ignoring. If you want to see it go away, just run the cell again.

In [3]:
import psycopg2

### First let's create a connection to the database

This connects to our local instance of PostgreSQL. From there we will use the databases/schema. This connection will reach out to the database and insure we have the correct privilages to connect to this database.

**Note 1: This block of code will be standard in all notebooks.**
    
**Note 2: Adding the try except will make sure errors are catch and understood**

In [5]:
try: 
    conn = psycopg2.connect("host='127.0.0.1' dbname='studentdb' user='postgres' ")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

### Next use that connect to get a cursor that we will use to execute queries.

In [6]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the Database")
    print(e)

**For this demo we will use automatic commit so that each action is commited without having to call conn.commit() after each command. The ability to rollback and commit transactions are a feature of Relational Databases.**

In [7]:
conn.set_session(autocommit=True)

### Let's test our connection and our error handling

We are trying to do a select * on a table we have not created yet. We should expect to see a nicely handled error.

In [9]:
try:
    cur.execute("select * from udacity.music_library")
except psycopg2.Error as e:
    print(e)

relation "udacity.music_library" does not exist
LINE 1: select * from udacity.music_library
                      ^



### Let's create a database to do our work in 

In [10]:
try: 
    cur.execute("create database udacity")
except psycopg2.Error as e:
    print(e)

Let's close our connection to the default database, reconnect to the Udacity database, and get a new cursor. 

In [11]:
try: 
    conn.close()
except psycopg2.Error as e:
    print(e)

try:
    conn = psycopg2.connect("dbname=udacity")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the Database")
    print(e)

conn.set_session(autocommit=True)

**Let's imagine we would like to start creating a Music Library of albums. Each album has a lot of information we could add to the music library table, but we will just start with album name, artist name, year. The type of table we want to create and the columns we need seems to fall out very naturally.**

`Table Name: music_library
column 1: Album Name
column 2: Artist Name
column 3: Year`

**Now to translate this information into a Create Table Statement.**

In [12]:
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS music_library (album_name varchar, artist_name varchar, year int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

**No error was found, but let's check to ensure our table was created. `select count(*)` which should return 0 as we have not inserted any rows.**

In [13]:
try: 
    cur.execute("select count(*) from music_library")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

print(cur.fetchall())

[(0,)]


### Let's insert two rows

Note the syntax here 

In [14]:
try:
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) \
                VALUES (%s, %s, %s)", \
               ("Let It Be", "The Beatles", 1970))
except psycopg2.Error as e:
    print("Error: Issue insert rows")
    print(e)

try:
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) \
                VALUES (%s, %s, %s)", \
               ("Rubber Soul", "The Beatles", 1965))
except psycopg2.Error as e:
    print("Error: Issue insert rows")
    print(e)



### Validate your data was inserted into the table
Note: The while loop is used for printing the results. If executing queries in the Postgres shell, this could not be required.

**Note: If you run the insert statement code more than once, you will see duplicates of your data. PostgreSQL allows for duplicates.**

In [15]:
try:
    cur.execute("select * from music_library;")
except psycopg.Error as e:
    print("Error: select *")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Let It Be', 'The Beatles', 1970)
('Rubber Soul', 'The Beatles', 1965)


**For the sake of the demo, I will drop the table.**

In [16]:
try:
    cur.execute("DROP table music_library")
except psycopg.Error as e:
    print("Error: Dropping table")
    print(e)

**And finally close your cursor and connection.**

In [17]:
cur.close()
conn.close()